In [ ]:
# Parameters cell for papermill launching
work_dir <- "./intermediate_branch_data/Nov-2023-log10"

In [ ]:
# Change to working directory
setwd(work_dir)

# Confirmation of change directory
getwd()

In [ ]:
system("conda env list", intern=TRUE)

In [ ]:
library(tidyverse)
# Capitalization matters!
library(ggExtra)
library(segmented)

## Scatter plot  
- using new merged dataset to create scatter plot
- add kernel densities to the plots (along the side)

In [ ]:
#some filtering/mutating first 
combods_split <- read_csv("ICON-ModEx_Combined_Predicted_Observed_Respiration_Rates.csv", na = c(-9999, "N/A", "NA", NA)) ### %>%
#remova Na's
  #split data set into "hot" and "cold" spots
  #TMP-TEST-SFG#mutate(ds_split = ifelse(Log_Observed_Normalized_Respiration_Rate > 2.7, "500+",                         ifelse(Log_Observed_Normalized_Respiration_Rate < 2.7, "500-", "500-") )) %>% 
  ###
### #clasifying by quadrant (hot, cold, misclass hot, misclass cold) by year
### # Need to get rid of the year because we are working year-by-year
###  mutate(ds_quadrant23 =                                                    ifelse(Log_Observed_Normalized_Respiration_Rate > 2.7 & Log_Predicted_Normalized_Respiration_Rate > 2.7 , "hs23",
###  ifelse(Log_Observed_Normalized_Respiration_Rate > 2.7 & Log_Predicted_Normalized_Respiration_Rate < 2.7 , "misclass_hs23",                                                                                              
### ifelse(Log_Observed_Normalized_Respiration_Rate < 2.7 & Log_Predicted_Normalized_Respiration_Rate < 2.7 , "cs23",    
### ifelse(Log_Observed_Normalized_Respiration_Rate < 2.7 & Log_Predicted_Normalized_Respiration_Rate > 2.7 , "misclass_cs23", NA)))) )

## Scatter plot in log-log space
 - needs to have kernel density plots on y and x axis
 - needs use log data
 - needs to have overall line, line for above 500, below 500, and R^2 values for all three
 - facet wrap or color dots by iteration (first versus last)
 
 - right now s19s iteration, but want OG 0th iteration

In [ ]:
#add in lm's
#run 3 lm up here, then extract R^2, then call into annotate 
lm_all_final <- lm(Log_Predicted_Normalized_Respiration_Rate ~ Log_Observed_Normalized_Respiration_Rate, combods_split)
summary(lm_all_final)

#breakpoint
breakpoint <- segmented(lm_all_final)$psi[,2]
bp_uncertainty <- segmented(lm_all_final)$psi[,3]
segCI <- confint(lm_all_final,level = 0.95)

combods_split <- combods_split%>%
   mutate(ds_split = ifelse(Log_Observed_Normalized_Respiration_Rate > breakpoint, "Hot", ifelse(Log_Observed_Normalized_Respiration_Rate < breakpoint, "Cold", "Cold") ))

lm_sup_final <- lm(Log_Predicted_Normalized_Respiration_Rate ~ Log_Observed_Normalized_Respiration_Rate, combods_split %>% filter(Log_Observed_Normalized_Respiration_Rate >= breakpoint))
summary(lm_sup_final)

lm_sub_final <- lm(Log_Predicted_Normalized_Respiration_Rate ~ Log_Observed_Normalized_Respiration_Rate, combods_split %>% filter(Log_Observed_Normalized_Respiration_Rate < breakpoint))
summary(lm_sub_final)

branch_name <- str_extract(work_dir, '[^/]+$')
branch_name_split <- str_split(branch_name, '-')[[1]]

# Relabel branch names for clarity
# Sep 2019 is completed 2019 data set, but it wasn't ready until very late 2021.
# Dec 2021 is data as presented to the community meeting with is an incomplete subset of Sep 2019 (now renamed).
if (str_detect(branch_name,"2019")){
    month <- "Dec"
    year <- "2021b"}else if(str_detect(branch_name,"2021")){
    month <- "Dec"
    year <- "2021a"
    }else{
    month <- branch_name_split[1]
    year <- branch_name_split[2]}

fullscatter_lastiteration <- 
  combods_split %>% ggplot(aes(abs(Log_Observed_Normalized_Respiration_Rate ),
                      abs(Log_Predicted_Normalized_Respiration_Rate) #)) + #
                      ,  color =  ds_split))+ 
#to see R^2 for above/below thresholds, remove color grouping above and use stat poly eq fx (below)
  geom_point(color = "darkgrey", size = 3, pch = 21) + 
    geom_vline (xintercept = breakpoint, linetype = "dashed", color="grey") + 
    geom_hline (yintercept = breakpoint, linetype = "dashed", color="dark grey") + #not a clear threshold 
      geom_abline (linetype = "longdash", color="black") + #not a clear threshold

# Required by ggextra, fails to import above. Where are the other lines (blue and red) plotted?
     #ggpmisc::stat_poly_line(color = "black", alpha = 0.5) + #overall best fit


# stat_poly_eq(use_label(c( "R2"))) + #remove hashtag to see R^2 of log500 split
 
    geom_smooth( aes(abs(Log_Observed_Normalized_Respiration_Rate ), #above/below threshold best fit
                      abs(Log_Predicted_Normalized_Respiration_Rate),
                      color =  ds_split), method = "lm", alpha = 0.2) + 
    geom_smooth( aes(abs(Log_Observed_Normalized_Respiration_Rate ), #above/below threshold best fit
                      abs(Log_Predicted_Normalized_Respiration_Rate)), method = "lm", alpha = 0.2,
                      color =  "black") +
  xlab("log10(Observed rates)") + 
  ylab("log10(Predicted rates)") + 
scale_shape_manual(values = c(21, 16)) + #shape or color by iteration
  scale_color_manual(values = c( "lightblue", "maroon")) +

annotate('text', label = str_c('R^2 == ', round(summary(lm_all_final)$r.squared, 2)), parse = T, x = 3.5, y = 0.75, size = 5, family = 'serif', color = "black")+
annotate('text', label = str_c('R^2 == ', round(summary(lm_sub_final)$r.squared, 2)), parse = T, x = 3.5, y = 0.5, size = 5, family = 'serif', color="lightblue")+
annotate('text', label = str_c('R^2 == ', round(summary(lm_sup_final)$r.squared, 2)), parse = T, x = 3.5, y = 0.25, size = 5, family = 'serif',  color="maroon")+

annotate('text', label = str_c('RMSE == ', round(sqrt(mean(lm_all_final$residuals^2)), 2)), parse = T, x = 2.0, y = 0.72, size = 5, family = 'serif', color = "black")+
annotate('text', label = str_c('RMSE == ', round(sqrt(mean(lm_sub_final$residuals^2)), 2)), parse = T, x = 2.0, y = 0.47, size = 5, family = 'serif', color = "lightblue")+
annotate('text', label = str_c('RMSE == ', round(sqrt(mean(lm_sup_final$residuals^2)), 2)), parse = T, x = 2.0, y = 0.22, size = 5, family = 'serif', color = "maroon")+

annotate('text', label = paste(month, year), x = 0.5, y = 4, size = 5, family = 'serif',  color="black")+

annotate('text', label = round(breakpoint,2), x = breakpoint, y = 3.8, size = 5, family = 'serif',  color="black")+

lims(y = c(0,4), x = c(0,4)) + 
  theme_classic() +
   theme(text = element_text(size = 16, family = 'serif'), 
         legend.position = 0)

#adding density plots on axes
 histograms <- ggExtra::ggMarginal(fullscatter_lastiteration, fill = "lightgrey")
 
# Original plot save in tidy
ggsave(filename = "./scatter.png", plot = histograms, width = 6, height = 6, dpi = 300)
ggsave(filename = "./scatter.pdf", plot = histograms, width = 6, height = 6, dpi = 300)
#ggsave(filename = "./scatter.pdf", plot = fullscatter_lastiteration, width = 6, height = 6, units="in", dpi = 300, device="pdf")  
# View plot inline
#histograms

# Alternative in base
#png("./scatter.png")
#print(histograms)
#print(fullscatter_lastiteration)
#dev.off()

In [ ]:
# Display figure inline if necessary
histograms

## Observations vs. predicted values scatter plots in normal space

In [ ]:
#add in lm's
# Linear model is what generates the R^2
#run 3 lm up here, then extract R^2, then call into annotate 
lm_all_final <- lm(Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_pre_avg ~ Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_obs, combods_split)
summary(lm_all_final)

#breakpoint
breakpoint <- segmented(lm_all_final)$psi[,2]
bp_uncertainty <- segmented(lm_all_final)$psi[,3]
segCI <- confint(lm_all_final,level = 0.95)

combods_split <- combods_split%>%
   mutate(ds_split = ifelse(Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_obs > breakpoint, "Hot", ifelse(Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_obs < breakpoint, "Cold", "Cold") ))


lm_sup_final <- lm(Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_pre_avg ~ Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_obs, combods_split %>% filter(Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_obs >= breakpoint))
summary(lm_sup_final)

lm_sub_final <- lm(Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_pre_avg ~ Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_obs, combods_split %>% filter(Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_obs < breakpoint))
summary(lm_sub_final)

fullscatter_lastiteration_no_log <- 
  combods_split %>% ggplot(aes(abs(Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_obs ),
                      abs(Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_pre_avg) #)) + #
                      ,  color =  ds_split))+ 
#to see R^2 for above/below thresholds, remove color grouping above and use stat poly eq fx (below)
  geom_point(color = "darkgrey", size = 3, pch = 21) + 
    geom_vline (xintercept = abs(breakpoint), linetype = "dashed", color="grey") + 
    geom_hline (yintercept = abs(breakpoint), linetype = "dashed", color="dark grey") + #not a clear threshold 
      geom_abline (linetype = "longdash", color="black") + #not a clear threshold

# Required by ggextra, fails to import above. Where are the other lines (blue and red) plotted?
     #ggpmisc::stat_poly_line(color = "black", alpha = 0.5) + #overall best fit


# stat_poly_eq(use_label(c( "R2"))) + #remove hashtag to see R^2 of log500 split
 
    geom_smooth( aes(abs(Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_obs ), #above/below threshold best fit
                      abs(Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_pre_avg),
                      color =  ds_split), method = "lm", alpha = 0.2) + 
    geom_smooth( aes(abs(Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_obs ), #above/below threshold best fit
                      abs(Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_pre_avg)), method = "lm", alpha = 0.2,
                      color =  "black") +
  xlab("Observed rates") + 
  ylab("Predicted rates") + 
scale_shape_manual(values = c(21, 16)) + #shape or color by iteration
  scale_color_manual(values = c( "lightblue", "maroon")) +

annotate('text', label = str_c('R^2 == ', round(summary(lm_all_final)$r.squared, 2)), parse = T, x = 3500, y = 1500, size = 5, family = 'serif', color = "black")+
annotate('text', label = str_c('R^2 == ', round(summary(lm_sub_final)$r.squared, 2)), parse = T, x = 3500, y = 1200, size = 5, family = 'serif', color="lightblue")+
annotate('text', label = str_c('R^2 == ', round(summary(lm_sup_final)$r.squared, 2)), parse = T, x = 3500, y = 1000, size = 5, family = 'serif',  color="maroon")+
    
annotate('text', label = str_c('RMSE == ', round(sqrt(mean(lm_all_final$residuals^2)), 2)), parse = T, x = 1000.0, y = 3500, size = 5, family = 'serif', color = "black")+
annotate('text', label = str_c('RMSE == ', round(sqrt(mean(lm_sub_final$residuals^2)), 2)), parse = T, x = 1000.0, y = 3200, size = 5, family = 'serif', color = "lightblue")+
annotate('text', label = str_c('RMSE == ', round(sqrt(mean(lm_sup_final$residuals^2)), 2)), parse = T, x = 1000.0, y = 3000, size = 5, family = 'serif', color = "maroon")+

annotate('text', label = paste(month, year), x = 450.0, y = 4000.0, size = 5, family = 'serif',  color="black")+

annotate('text', label = round(abs(breakpoint),2), x = abs(breakpoint), y = 3800.0, size = 5, family = 'serif',  color="black")+

  lims(y = c(0,4000), x = c(0,4000)) + 
  theme_classic() +
   theme(text = element_text(size = 16, family = 'serif'), 
         legend.position = 0)

#adding density plots on axes
 histograms_no_log <- ggExtra::ggMarginal(fullscatter_lastiteration_no_log, fill = "lightgrey")
 
# Original plot save in tidy
ggsave(filename = "./scatter_no_log.png", plot = histograms_no_log, width = 6, height = 6, dpi = 300)  

In [ ]:
#fullscatter_lastiteration_no_log
histograms_no_log